In [148]:
import numpy as np
import nibabel as nib
from nibabel.affines import apply_affine
import matplotlib.pyplot as plt

In [149]:
# Load data
# Load right hemisphere occipital region 200um 
rhOccip = nib.load('raw_data/native_RHoccip_200um.nii.gz')
rhOccipData = rhOccip.get_fdata()  # extract voxel array
print(f"RH volume shape: {rhOccipData.shape}")

# Load crop of occip ROI (cropped from rh 200um volume)
occipCrop = nib.load('raw_data/RHoccip_crop_square_200um.nii.gz')
occipCropData = occipCrop.get_fdata()
print(f"RH Occip crop volume shape: {occipCropData.shape}")

RH volume shape: (356, 512, 355)
RH Occip crop volume shape: (90, 90, 90)


In [150]:
# Slice from original (end exclusive)
x_start, x_end = 239, 329  # sagittal
y_start, y_end = 202, 292  # axial
z_start, z_end = 108, 198  # coronal

In [151]:
# affine transforms
# Full brain 100um affine matrix (assumed source of freesurfer surfaces but not sure)
full_affine = np.array([[-0.1, 0., 0., 80.0],
                         [0., 0., 0.1, -78.05],
                         [0., -0.1, 0., 71.03],
                         [0., 0., 0., 1.]])

# offset from full brain to RH crop in 100um voxel space
offset = np.array([263, 50, 0])

# cortical depth values (0.0 = pial, 1.0 = White)
depths = np.arange(0, 1.025, 0.025)
print(f"{len(depths)} cortical depth layers")

41 cortical depth layers


In [152]:
if False: # cras from surface metadata

    coords, faces, volume_info = nib.freesurfer.read_geometry(
        'raw_data/FreeSurfer_Surfaces/rh.equi400.5.pial',
        read_metadata=True
    )

    # nibabel SurfaceRAS to voxel using metadata
    from nibabel.freesurfer import MGHImage

    # Extract the transformation info
    print("Volume info keys:", volume_info.keys())

    # metadata to confirm source of surfaces and cras

    volume_shape = np.array(volume_info['volume'])  # [1760, 1280, 1760]
    voxelsize = np.array(volume_info['voxelsize'])  # [0.1, 0.1, 0.1]
    xras = np.array(volume_info['xras'])  # [-1, 0, 0]
    yras = np.array(volume_info['yras'])  # [0, 0, -1]
    zras = np.array(volume_info['zras'])  # [0, 1, 0]
    cras = np.array(volume_info['cras'])  # [-8.0, 9.94999695, 7.03147888]

    print(cras)



In [153]:
if False: # find valid vertex coords that fall into cropped volume

    all_voxel_coords = []  # voxel coordinates for each depth layer
    all_masks = []  # validity masks for each depth layer
    cras = np.array([-8.0, 9.95, 7.0315])

    for depth in depths:
        # Construct filename for this depth layer
        surf_file = f'raw_data/FreeSurfer_Surfaces/rh.equi_test{depth:.1f}.pial'
        
        # Load surface vertex coordinates (RAS space, mm) and face connectivity
        coords, faces = nib.freesurfer.read_geometry(surf_file)

        scanner_ras = coords + cras
        
        # Transform: Full brain RAS (mm) -> Full brain voxels (100um)
        voxels_fullbrain = apply_affine(np.linalg.inv(full_affine), scanner_ras)
        
        # Apply offset: Full brain voxels -> RH crop voxels (100um)
        voxels_rh_100um = voxels_fullbrain - offset
        
        # Scale: RH crop voxels (100um) -> RH crop voxels (200um)
        voxels_rh_200um = voxels_rh_100um / 2.0
        
        # Check which vertices fall within the RH 200um volume bounds
        inCrop = ((voxels_rh_200um[:, 0] >= 0) & (voxels_rh_200um[:, 0] < rhOccipData.shape[0]) &
                (voxels_rh_200um[:, 1] >= 0) & (voxels_rh_200um[:, 1] < rhOccipData.shape[1]) &
                (voxels_rh_200um[:, 2] >= 0) & (voxels_rh_200um[:, 2] < rhOccipData.shape[2]))
        
        # Store results for this depth layer
        all_masks.append(inCrop)
        all_voxel_coords.append(voxels_rh_200um)
        
        print(f"Depth {depth:.2f}: {inCrop.sum()} vertices in bounds")

    # Convert lists to numpy arrays for easier manipulation
    all_masks = np.array(all_masks)  # Shape: (21, N_vertices)
    all_voxel_coords = np.array(all_voxel_coords)  # Shape: (21, N_vertices, 3)




In [154]:
# filter for smaller crop
all_voxel_coords = []
all_masks = []
cras = np.array([-8.0, 9.95, 7.0315])
for depth in depths:
    surf_file = f'raw_data/FreeSurfer_Surfaces/rh.equi40{depth:.1f}.pial'
    coords, faces = nib.freesurfer.read_geometry(surf_file)
    scanner_ras = coords + cras
    voxels_fullbrain = apply_affine(np.linalg.inv(full_affine), scanner_ras)
    voxels_rh_100um = voxels_fullbrain - offset
    voxels_rh_200um = voxels_rh_100um / 2.0
    
    # Small crop bounds + offset to crop coordinates
    inCrop = ((voxels_rh_200um[:, 0] >= 239) & (voxels_rh_200um[:, 0] < 329) &
              (voxels_rh_200um[:, 1] >= 202) & (voxels_rh_200um[:, 1] < 292) &
              (voxels_rh_200um[:, 2] >= 108) & (voxels_rh_200um[:, 2] < 198))
    
    # Offset to crop-relative coordinates
    voxels_rh_200um[:, 0] -= 239
    voxels_rh_200um[:, 1] -= 202
    voxels_rh_200um[:, 2] -= 108
    
    all_masks.append(inCrop)
    all_voxel_coords.append(voxels_rh_200um)

all_masks = np.array(all_masks)
all_voxel_coords = np.array(all_voxel_coords)

In [155]:
# filter only full sets of depth

# vertices that are within bounds across all depth layers
valid_all_layers = np.all(all_masks, axis=0)  # axis=0 check across depth dimension

print(len(valid_all_layers))
n_valid = valid_all_layers.sum()
n_total = len(valid_all_layers)
print(f"\nVertices valid all {len(depths)} layers: {n_valid} out of {n_total} ({100*n_valid/n_total:.1f}%)")

249329

Vertices valid all 41 layers: 3343 out of 249329 (1.3%)


In [156]:
# extract intensity profiles

# array to store intensity profiles
# Rows = valid vertices, Columns = depth layers
intensity_profiles = np.zeros((n_valid, len(depths)))

# Extract voxel coordinates for valid vertices only
valid_coords = all_voxel_coords[:, valid_all_layers, :]  # Shape: (21, n_valid, 3)

# Loop through each depth layer to sample intensities
for depth_idx in range(len(depths)):
    # Get voxel coordinates for this depth layer
    voxels = valid_coords[depth_idx]  # Shape: (n_valid, 3)
    
    # Convert continuous coordinates to integer voxel indices
    voxel_indices = np.floor(voxels).astype(int)
    
    # Sample intensities from volume at these voxel locations
    intensities = occipCropData[voxel_indices[:, 0], voxel_indices[:, 1], voxel_indices[:, 2]]
    
    # Store in profile array\
    intensity_profiles[:, depth_idx] = intensities

print(f"extracted profiles with shape: {intensity_profiles.shape}")
print(f"Intensity range: {intensity_profiles.min():.2f} to {intensity_profiles.max():.2f}")

extracted profiles with shape: (3343, 41)
Intensity range: 154.76 to 3530.31


In [157]:
import ipywidgets as widgets
from IPython.display import display
# No. of profiles to display
n_profiles_per_view = 3

# interactive plotting
def plot_profiles(start_index):
    """
    Plot batch of cortical depth profiles starting from start_index
    """
    # Calculate end index for this batch
    end_index = min(start_index + n_profiles_per_view, n_valid)
    
    # Select depth range to plot
    depth_start_idx = 3
    depth_end_idx = 38  
    depths_to_plot = depths[depth_start_idx:depth_end_idx]
    
    
    plt.figure(figsize=(12, 8))
    
    # Plot each profile in the current batch
    for idx in range(start_index, end_index):
        profile = intensity_profiles[idx, depth_start_idx:depth_end_idx]
        plt.plot(depths_to_plot, profile, alpha=0.7, linewidth=1.5, label=f'Vertex {idx}')
    
    # plot stuff
    plt.xlabel('Cortical Depth (0 = pial, 1 = WM)', fontsize=12)
    plt.ylabel('Intensity', fontsize=12)
    plt.title(f'Cortical Depth Intensity Profiles (Vertices {start_index} to {end_index-1} depths {depth_start_idx}:{depth_end_idx})', fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [158]:
# Create slider widget
# range 0 to (n_valid - n_profiles_per_view) in steps of n_profiles_per_view
widgets.interact(
    plot_profiles,
    start_index=widgets.IntSlider(
        value=0,  # Initial position
        min=0,  # Start at vertex 0
        max=n_valid - n_profiles_per_view,  # End when fewer than 10 profiles remain
        step=n_profiles_per_view,  # stepsize for slider
        description='profiles:',  # slider label
        continuous_update=False  
    )
)

# summary statistics
print(f"\nProfile Statistics:")
print(f"Total valid vertices: {n_valid}")
print(f"Profiles per view: {n_profiles_per_view}")
print(f"Total batches: {int(np.ceil(n_valid / n_profiles_per_view))}")
print(f"Mean intensity across all profiles: {intensity_profiles.mean():.2f}")
print(f"Std intensity across all profiles: {intensity_profiles.std():.2f}")

interactive(children=(IntSlider(value=0, continuous_update=False, description='profiles:', max=3340, step=3), …


Profile Statistics:
Total valid vertices: 3343
Profiles per view: 3
Total batches: 1115
Mean intensity across all profiles: 2116.56
Std intensity across all profiles: 372.51


In [ ]:
# random vertex for visualisation checking
test_vertex_id = 6349

# Get its position at middle depth - 0.5
coords_at_middepth, _ = nib.freesurfer.read_geometry('raw_data/FreeSurfer_Surfaces/rh.equi400.5.pial')
ras_position = coords_at_middepth[test_vertex_id]
print(f"Vertex {test_vertex_id} at middle depth:")
print(f"  Surface RAS position: {ras_position}")

# add CRAS
cras = np.array([-8.0, 9.95, 7.0315])
scanner_ras = ras_position + cras
print(f"  Scanner RAS position: {scanner_ras}")

# Use RH 200um affine
rh_affine = rhOccip.affine
voxels_rh_200um = apply_affine(np.linalg.inv(rh_affine), scanner_ras.reshape(1, 3))
voxel_idx_rh = np.floor(voxels_rh_200um).astype(int)

print(f"  Calculated voxel: {voxel_idx_rh[0]}")
print(f"  Freeview: [277, 253, 127]")
print(f"  Sampled intensity: {rhOccipData[voxel_idx_rh[0,0], voxel_idx_rh[0,1], voxel_idx_rh[0,2]]}")

Vertex 6349 at middle depth:
  Surface RAS position: [  6.28624773 -62.62726593   8.4480381 ]
  Scanner RAS position: [ -1.71375227 -52.67726593  15.4795381 ]
  Calculated voxel: [277 252 126]
  Freeview: [277, 253, 127]
  Sampled intensity: 2100.50554541886


In [ ]:
if False:
    coords, faces = nib.freesurfer.read_geometry('raw_data/FreeSurfer_Surfaces/rh.equi_test0.5.pial')
    # Load FreeSurfer surface

    nib.freesurfer.write_geometry('raw_data/FreeSurfer_Surfaces/rh.equi_test0.5.vtk', 
                                coords, faces)
    # Save as VTK format